# 04-deployment

We'll deploy the ride duration model in `batch` mode, and we will use the **Yellow Taxi Trip Records** dataset.

## Q1. Notebook

What's the standard deviation of the predicted duration for the dataset of **March 2023**.

In [10]:
import pickle
import pandas as pd
import numpy as np

In [2]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [4]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [12]:
year = '2023'
month = '03'
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month}.parquet')

In [9]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [11]:
np.std(y_pred)

np.float64(6.247488852238703)

## Q2. Preparing the output

First, let's create an artificial `ride_id` column, then, write the `ride_id` and the `predictions` to a dataframe with results and store it as parquet datafile.

In [24]:
year = 2023
month = 3
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [28]:
df_result = df[['ride_id']].copy()
df_result['predictions'] = y_pred

In [30]:
output_file = f'predictions/predictions_yellow_tripdata_{year:04d}-{month:02d}.parquet'
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [31]:
!ls -lh {output_file}

-rw-r--r--. 1 ltruciosr-dev ltruciosr-dev 66M Jun 28 13:48 predictions/predictions_yellow_tripdata_2023-03.parquet


## Q3. Creating the scoring script

Now let's put everything into a virtual environment by creating two files `Pipfile` and `Pipfile.lock`. 

The `Pipfile.lock` file keeps the hashes of the dependencies we use for the virtual env.

Follow this steps:

1. Create a new pipenv inside the project

```bash
    pipenv --python 3.x
```

2. Add dependencies and modify according to the current project

```bash
    [[source]]
    url = "https://pypi.org/simple"
    verify_ssl = true
    name = "pypi"

    [packages]
    scikit-learn = "==1.5.0"
    pandas = "==2.2.2"

    [dev-packages]

    [requires]
    python_version = "3.10"
    python_full_version = "3.10.14"

```

3. Generate the lock file

```bash
    pipenv lock
```